### Drop out

In [0]:
import tensorflow as tf
X = [1.5, 0.5, 0.75, 1.0, 0.75, 0.6]
p_keep = 0.5
# p_keep = 2/6
drop_out = tf.nn.dropout(X, p_keep)

In [0]:
sess = tf.Session()
sess.run(drop_out)

array([3. , 1. , 1.5, 2. , 0. , 0. ], dtype=float32)

### Tensorflow on Mnist

In [0]:
# Step 0: Loading and Preprocess Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist", one_hot=True,reshape=False,validation_size=0)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models

In [0]:
import tensorflow as tf

In [0]:
# Step1: Initial Setup
X = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,10])

In [0]:
L1= 16
L2 = 32
L3 = 64 

In [0]:
W1 = tf.Variable(tf.truncated_normal([3,3,1,L1],stddev=0.1))
B1 = tf.Variable(tf.truncated_normal([L1],stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([3,3,L1,L2],stddev=0.1))
B2 = tf.Variable(tf.truncated_normal([L2],stddev=0.1))

W3 = tf.Variable(tf.truncated_normal([7*7*L2,L3],stddev=0.1))
B3 = tf.Variable(tf.truncated_normal([L3],stddev=0.1))

W4 = tf.Variable(tf.truncated_normal([L3,10],stddev=0.1))
B4 = tf.Variable(tf.truncated_normal([10],stddev=0.1))


In [0]:
# Step 2 Model
Y1 = tf.nn.relu(tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')+B1)
Y1 = tf.nn.max_pool(Y1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
Y2 = tf.nn.relu(tf.nn.conv2d(Y1,W2,strides=[1,1,1,1],padding='SAME')+B2)
Y2 = tf.nn.max_pool(Y2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

YY = tf.reshape(Y2,shape=[-1,7*7*L2])

Y3 = tf.nn.relu(tf.matmul(YY,W3)+B3)

In [0]:
Ylogits = tf.matmul(Y3,W4)+B4
yhat = tf.nn.softmax(Ylogits)

In [0]:
# Step 3 Loss Function

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=Ylogits))

In [0]:
# Step 4 Optimizer

lr = 0.01
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [0]:
# accuracy of the trained model
is_correct = tf.equal(tf.argmax(y,1),tf.argmax(yhat,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

training_epochs = 2
batch_size = 100

# Step 5: Training Loop
for epoch in range(training_epochs):
    num_batches = int(mnist.train.num_examples / batch_size)
    for i in range(num_batches):
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        train_data = {X: batch_X, y: batch_y}
        sess.run(train, feed_dict=train_data)
        if i%100 == 0:
            print(epoch * num_batches + i + 1, "Training accuracy =", sess.run(accuracy, feed_dict=train_data),
          "Loss =", sess.run(loss, feed_dict=train_data))


1 Training accuracy = 0.13 Loss = 2.2889564
101 Training accuracy = 0.94 Loss = 0.122412115
201 Training accuracy = 0.97 Loss = 0.054826837
301 Training accuracy = 0.97 Loss = 0.09395942
401 Training accuracy = 0.96 Loss = 0.08869322
501 Training accuracy = 0.99 Loss = 0.07872682
601 Training accuracy = 0.98 Loss = 0.10903137
701 Training accuracy = 1.0 Loss = 0.0042077047
801 Training accuracy = 0.99 Loss = 0.03129769
901 Training accuracy = 0.98 Loss = 0.062494073
1001 Training accuracy = 1.0 Loss = 0.0143799195
1101 Training accuracy = 0.99 Loss = 0.01822922


In [0]:
test_data = {X:mnist.test.images,y:mnist.test.labels}
print("Testing Accuracy = ", sess.run(accuracy, feed_dict = test_data))

Testing Accuracy =  0.986


In [0]:
### Ex: CNN on CIFAR

In [0]:
# Step0: Load data
from tensorflow.python.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [36]:
X_train.shape

(50000, 32, 32, 3)

In [37]:
y_train.shape

(50000, 1)

In [38]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [39]:
y_train.flatten()

array([6, 9, 9, ..., 9, 1, 1], dtype=uint8)

In [0]:
import numpy as np
num_labels = len(np.unique(y_train))
y_train = np.eye(num_labels)[y_train.flatten()]

In [41]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [42]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [43]:
X_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [0]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [0]:
# Step1: Initial Setup
X = tf.placeholder(tf.float32,[None,32,32,3])
y = tf.placeholder(tf.float32,[None,10])

In [0]:
L1= 32
L2 = 64
L3 = 128

In [0]:
W1 = tf.Variable(tf.truncated_normal([3,3,3,L1],stddev=0.1))
B1 = tf.Variable(tf.truncated_normal([L1],stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([3,3,L1,L2],stddev=0.1))
B2 = tf.Variable(tf.truncated_normal([L2],stddev=0.1))

W3 = tf.Variable(tf.truncated_normal([8*8*L2,L3],stddev=0.1))
B3 = tf.Variable(tf.truncated_normal([L3],stddev=0.1))

W4 = tf.Variable(tf.truncated_normal([L3,10],stddev=0.1))
B4 = tf.Variable(tf.truncated_normal([10],stddev=0.1))

In [0]:
# Step 2 Build Model
Y1 = tf.nn.relu(tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')+B1)
Y1 = tf.nn.max_pool(Y1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
Y2 = tf.nn.relu(tf.nn.conv2d(Y1,W2,strides=[1,1,1,1],padding='SAME')+B2)
Y2 = tf.nn.max_pool(Y2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

YY = tf.reshape(Y2,shape=[-1,8*8*L2])

Y3 = tf.nn.relu(tf.matmul(YY,W3)+B3)

In [0]:
Ylogits = tf.matmul(Y3,W4)+B4
yhat = tf.nn.softmax(Ylogits)

In [0]:
# Step 3 Loss Function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=Ylogits))

In [0]:
# Step 4 Optimizer
lr = 0.01
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [55]:
# accuracy of the trained model
is_correct = tf.equal(tf.argmax(y,1),tf.argmax(yhat,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

training_epochs = 2
batch_size = 100

# Step 5: Training Loop
for epoch in range(training_epochs):
    num_batches = int(X_train.shape[0] / batch_size)
    for i in range(num_batches):
        batch_X, batch_y = X_train[ i*batch_size : (i+1)*batch_size ], y_train[ i*batch_size : (i+1)*batch_size ]
        train_data = {X: batch_X, y: batch_y}
        sess.run(train, feed_dict=train_data)
        if i%100 == 0:
            print(epoch * num_batches + i + 1, "Training accuracy =", sess.run(accuracy, feed_dict=train_data),
              "Loss =", sess.run(loss, feed_dict=train_data))

1 Training accuracy = 0.12 Loss = 6.303515
101 Training accuracy = 0.51 Loss = 1.667814
201 Training accuracy = 0.39 Loss = 1.6976204
301 Training accuracy = 0.54 Loss = 1.4256012
401 Training accuracy = 0.59 Loss = 1.1490167
501 Training accuracy = 0.5 Loss = 1.3918757
601 Training accuracy = 0.6 Loss = 1.1987575
701 Training accuracy = 0.53 Loss = 1.379945
801 Training accuracy = 0.54 Loss = 1.2597442
901 Training accuracy = 0.68 Loss = 0.95833474


In [0]:
y_test=np.eye(num_labels)[y_test.flatten()]

In [57]:
test_data = {X:X_test,y:y_test}
print("Testing Accuracy = ", sess.run(accuracy, feed_dict = test_data))

Testing Accuracy =  0.5496
